[View in Colaboratory](https://colab.research.google.com/github/ebgv/Planet--Understanding-the-Amazon-from-Space/blob/master/building_simple_model.ipynb)

In [8]:
# check the environment 

%ls

data/  sample_data/


In [2]:
'''# check if the data is in the environment 

%cd /content/data/sample/test/cloudy
%ls'''

'# check if the data is in the environment \n\n%cd /content/data/sample/test/cloudy\n%ls'

In [3]:
'''# check if the data is in the environment 

%cd /content/data/sample/train/cloudy
%ls'''

'# check if the data is in the environment \n\n%cd /content/data/sample/train/cloudy\n%ls'

# Imports 

In [0]:
# to install pytorch on colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [10]:
!pip install -U bcolz

Requirement already up-to-date: bcolz in /usr/local/lib/python3.6/dist-packages (1.2.1)


In [11]:
!pip install Pillow==4.0.0

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import bcolz
import time

# GPU Settings 

In [13]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

dtype = torch.FloatTensor
if use_gpu:
    dtype = torch.cuda.FloatTensor

Using gpu: True 


# Data Loader

In [0]:
# loader taken from class example: parameters to verify

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # statistics from imagenet ? 

prep1 = transforms.Compose([
                transforms.CenterCrop(224), # default cropping 
                transforms.ToTensor(),
                normalize,
            ])

In [0]:
data_dir = '/content/data/sample'
#data_dir = '/content/data'

#batch_size = 4
batch_size = 64

In [0]:
# dictionary of datasets - in this case only train data set 
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), prep1)
         for x in ['train', 'test']}

In [17]:
# train labels 
print(dsets['train'].imgs[:5]) # 5 first images and labels 
print(dsets['train'].imgs[len(dsets['train'])-5:]) # 5 last images and labels 

[('/content/data/sample/train/clear/train_10002.jpg', 0), ('/content/data/sample/train/clear/train_10095.jpg', 0), ('/content/data/sample/train/clear/train_10190.jpg', 0), ('/content/data/sample/train/clear/train_10194.jpg', 0), ('/content/data/sample/train/clear/train_10226.jpg', 0)]
[('/content/data/sample/train/partly_cloudy/train_9827.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9835.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9840.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_994.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9992.jpg', 3)]


In [18]:
# test labels 
print(dsets['test'].imgs[:5]) # 5 first images and labels 
print(dsets['test'].imgs[len(dsets['test'])-5:]) # 5 last images and labels 

[('/content/data/sample/test/clear/train_10215.jpg', 0), ('/content/data/sample/test/clear/train_10266.jpg', 0), ('/content/data/sample/test/clear/train_10272.jpg', 0), ('/content/data/sample/test/clear/train_10355.jpg', 0), ('/content/data/sample/test/clear/train_10368.jpg', 0)]
[('/content/data/sample/test/partly_cloudy/train_9347.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9578.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9641.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9851.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9986.jpg', 3)]


In [19]:
dset_classes = dsets['train'].classes
dset_classes # binary classes to begin 

['clear', 'cloudy', 'haze', 'partly_cloudy']

In [0]:
# dictionary of data loaders - again only train for now 
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=True, num_workers=0)
                for x in ['train', 'test']}

In [23]:
print(len(dsets['train']))
print(len(dsets['test']))

6400
1600


# Model: simple classifier from scratch

In [0]:
# need to understand parameters in more details !!!!

import torch.nn as nn
import torch.nn.functional as F # other way to define layers: when no need to update parameters 

# max pooling layer is not in init: no parameters to update ie. deterministic function  

class classifier(nn.Module):
    
    def __init__(self):
        super(classifier, self).__init__() # do the init from the inherited class
        # then what are the parameters of the network: need to be defined in init 
        # fill the missing entries below
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1) # check meaning of arguments  
        self.fc = nn.Linear(in_features=32*32*64, out_features=4) # 32 = 224 / 7
        
    def forward(self,x):
        # implement your network here, use F.max_pool2d, F.log_softmax and do not forget to flatten your vector
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size=7, stride=7)
        x = x.view(-1, 32*32*64) # flatten 
        x = self.fc(x)
        return F.log_softmax(x, dim=1) # softmax across the line ! not the component 
    
# since we inherit from nn module, the forward is called by default (do not call classifier.forward() !!

# Training Framework

In [0]:
def train_model(model,data_loader,loss_fn,optimizer,n_epochs=1):
    
    model.train(True)
    
    loss_train = np.zeros(n_epochs)
    acc_train = np.zeros(n_epochs)
    
    for epoch_num in range(n_epochs):
        running_corrects = 0.0
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, labels = data
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()    
                
            # batch_size ?
            bs = labels.size(0)
            
            # define the loss to minimize
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            # define the optimizer
            optimizer = optimizer
            optimizer.zero_grad()
            # backpropagation
            loss.backward()
            optimizer.step()
            
            # predictions to get statistics 
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == labels.data)
            size += bs
        # epoch statistics     
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.item() / size
        loss_train[epoch_num] = epoch_loss
        acc_train[epoch_num] = epoch_acc
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
    return loss_train, acc_train

# Running training epochs

# Using the simple classfier 

In [26]:
# using the model from scratch

# instanciate the model 
conv_class = classifier()
use_gpu = torch.cuda.is_available()
if use_gpu:
    conv_class = conv_class.cuda()

# choose the appropriate loss
loss_fn = torch.nn.CrossEntropyLoss()
# learning rate 
learning_rate = 1e-3
# your SGD optimizer
optimizer_cl = torch.optim.Adam(conv_class.parameters(), lr=learning_rate)


# and train for 10 epochs
l_t, a_t = train_model(conv_class, dset_loaders['train'], loss_fn, optimizer_cl, n_epochs = 20)

Train - Loss: 0.0269 Acc: 0.5914
Train - Loss: 0.0131 Acc: 0.6366
Train - Loss: 0.0129 Acc: 0.6506
Train - Loss: 0.0125 Acc: 0.6633
Train - Loss: 0.0117 Acc: 0.6866
Train - Loss: 0.0112 Acc: 0.7009
Train - Loss: 0.0116 Acc: 0.6942
Train - Loss: 0.0107 Acc: 0.7241
Train - Loss: 0.0102 Acc: 0.7383
Train - Loss: 0.0101 Acc: 0.7370
Train - Loss: 0.0096 Acc: 0.7602
Train - Loss: 0.0094 Acc: 0.7625
Train - Loss: 0.0089 Acc: 0.7777
Train - Loss: 0.0086 Acc: 0.7856
Train - Loss: 0.0081 Acc: 0.8100
Train - Loss: 0.0081 Acc: 0.8048
Train - Loss: 0.0078 Acc: 0.8203
Train - Loss: 0.0078 Acc: 0.8098
Train - Loss: 0.0073 Acc: 0.8277
Train - Loss: 0.0069 Acc: 0.8491


In [27]:
# and train for a few more epochs
l_t, a_t = train_model(conv_class, dset_loaders['train'], loss_fn, optimizer_cl, n_epochs = 5)

Train - Loss: 0.0065 Acc: 0.8547
Train - Loss: 0.0062 Acc: 0.8714
Train - Loss: 0.0062 Acc: 0.8658
Train - Loss: 0.0060 Acc: 0.8692
Train - Loss: 0.0056 Acc: 0.8781


# Testing 

In [0]:
def test(model,data_loader):
    model.train(False)

    running_corrects = 0.0
    running_loss = 0.0
    size = 0

    for data in data_loader:
        inputs, labels = data
        if use_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
            
        bs = labels.size(0)
                
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        _,preds = torch.max(outputs.data,1)
        
        # statistics
        running_loss += loss.data.item()
        running_corrects += torch.sum(preds == labels.data)
        size += bs

    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.item() / size))

In [29]:
# using simple classifier 

test(conv_class, dset_loaders['test'])

Test - Loss: 0.0155 Acc: 0.6675
